# Group work

1. Erniyaz Ashuov
2. Tolegen Aiteni
3. Daulet Akhmetbekov

Explanation what we understand:

Dense
[1 0 0 0 0
 0 0 0 5 0
 0 0 0 0 0
 0 2 0 0 0
 0 0 0 0 3]

Количество элементов: 25
Ненулевых: всего 4

Sparse
(0,0) = 1
(1,3) = 5
(3,1) = 2
(4,4) = 3

row =    [0, 1, 3, 4]
column = [0, 3, 1, 4]
value =  [1, 5, 2, 3]


Потому что если матрица 10 000 * 10 000 содержит 99% нулей:

плотная матрица (dense) -> 100 млн элементов

разреженная sparse - всего ~ 1000 элементов


зачем нужен

экономит память

ускоряет вычисления

позволяет работать с огромными матрицами, которые иначе не поместились бы в RAM


Обычная матрица N×N требует хранения N^2 элементов.

Например:

10000 * 10000 = 100 000 000 элементов ≈ 800 МБ (если double)

Sparse (если ненулевых только 20 000):

20 000 элементов + индексы ≈ 0.5 – 1.5 МБ Разница — в сотни раз


В sparse матрице:

операции просто пропускают нули

вычисления идут только по ненулевым индексам






---



**COO (Coordinate Matrix)**

**CSR (Compressed Sparse Row)**

**CSC (Compressed Sparse Column)**


---------------
COO Format (Coordinate List)

(rows, columns, values)

[0 0 5

 7 0 0

 0 0 1]

row    = [0, 1, 2]

col    = [2, 0, 2]



value  = [5, 7, 1]


+ простой

+ легко создавать

+ хорош для загрузки данных


- медленные операции

- плохо для умножения

-----------------------------



CSR Format (Compressed Sparse Row)

(rows, columns, values)

sparse vector * sparse matrix

[0 0 5

 7 0 0

 0 0 1]

values

[5, 7, 1]

col_index

[2, 0, 2]

row_ptr

[0, 1, 2, 3]




----------

CSC (Compressed Sparse Column)

row_ptr     = где начинается строка

col_index   = какие столбцы

values      = значения

A=[
  020​
  
  103​
  
  004​
​]

values    = [1, 2, 3, 4]

row_index = [1, 0, 1, 2]

col_ptr   = [0, 1, 2, 4]

start = col_ptr[2] = 2

end   = col_ptr[3] = 4

values[2] = 3, row_index[2] = 1  → A[1,2] = 3

values[3] = 4, row_index[3] = 2  → A[2,2] = 4


support multiplication sparse vector × sparse matrix



----------

Sparse Vector

v = [1, 0, 0, 0, 5, 0, 0, 3]


indices = [0, 4, 7]


values  = [1, 5, 3]

or

[(0, 1), (4, 5), (7, 3)]





---



#Dense → COO

In [ ]:
A = [
    [0, 0, 5],
    [7, 0, 0],
    [0, 0, 1]
]

def dense_to_coo(mat):
    rows = []
    cols = []
    values = []
    for i, row in enumerate(mat):
        for j, val in enumerate(row):
            if val != 0:
                rows.append(i)
                cols.append(j)
                values.append(val)
    return rows, cols, values

def coo_to_dense(rows, cols, values, n_rows, n_cols):
    mat = [[0 for _ in range(n_cols)] for _ in range(n_rows)]
    for r, c, v in zip(rows, cols, values):
        mat[r][c] = v
    return mat

# пример
rows, cols, vals = dense_to_coo(A)
print("COO представление")
print("rows   =", rows)
print("cols   =", cols)
print("values =", vals)

A_back = coo_to_dense(rows, cols, vals, 3, 3)
print("Восстановленная матрица из COO")
print(A_back)


COO представление
rows   = [0, 1, 2]
cols   = [2, 0, 2]
values = [5, 7, 1]
Восстановленная матрица из COO
[[0, 0, 5], [7, 0, 0], [0, 0, 1]]




---



#Dense → CSR

In [ ]:
def dense_to_csr(mat):
    values = []
    col_index = []
    row_ptr = [0]
    nnz_count = 0

    for row in mat:
        for j, val in enumerate(row):
            if val != 0:
                values.append(val)
                col_index.append(j)
                nnz_count += 1
        row_ptr.append(nnz_count)

    return values, col_index, row_ptr

def csr_to_dense(values, col_index, row_ptr, n_rows, n_cols):
    mat = [[0 for _ in range(n_cols)] for _ in range(n_rows)]
    for i in range(n_rows):
        start = row_ptr[i]
        end = row_ptr[i + 1]
        for p in range(start, end):
            j = col_index[p]
            mat[i][j] = values[p]
    return mat

# пример
csr_values, csr_cols, csr_row_ptr = dense_to_csr(A)
print("CSR представление:")
print("values   =", csr_values)
print("col_index=", csr_cols)
print("row_ptr  =", csr_row_ptr)

A_back_csr = csr_to_dense(csr_values, csr_cols, csr_row_ptr, 3, 3)
print("Восстановленная матрица из CSR:")
print(A_back_csr)


CSR представление:
values   = [5, 7, 1]
col_index= [2, 0, 2]
row_ptr  = [0, 1, 2, 3]
Восстановленная матрица из CSR:
[[0, 0, 5], [7, 0, 0], [0, 0, 1]]




---



#Dense → CSC

In [ ]:
def dense_to_csc(mat):
    n_rows = len(mat)
    n_cols = len(mat[0]) if n_rows > 0 else 0

    values = []
    row_index = []
    col_ptr = [0]
    nnz_count = 0

    for j in range(n_cols):
        for i in range(n_rows):
            val = mat[i][j]
            if val != 0:
                values.append(val)
                row_index.append(i)
                nnz_count += 1
        col_ptr.append(nnz_count)

    return values, row_index, col_ptr

def csc_to_dense(values, row_index, col_ptr, n_rows, n_cols):
    mat = [[0 for _ in range(n_cols)] for _ in range(n_rows)]
    for j in range(n_cols):
        start = col_ptr[j]
        end = col_ptr[j + 1]
        for p in range(start, end):
            i = row_index[p]
            mat[i][j] = values[p]
    return mat

# пример
csc_values, csc_rows, csc_col_ptr = dense_to_csc(A)
print("CSC представление:")
print("values    =", csc_values)
print("row_index =", csc_rows)
print("col_ptr   =", csc_col_ptr)

A_back_csc = csc_to_dense(csc_values, csc_rows, csc_col_ptr, 3, 3)
print("Восстановленная матрица из CSC:")
print(A_back_csc)


CSC представление:
values    = [7, 5, 1]
row_index = [1, 0, 2]
col_ptr   = [0, 1, 1, 3]
Восстановленная матрица из CSC:
[[0, 0, 5], [7, 0, 0], [0, 0, 1]]




---



#Sparse vector: dense_to_sparse, sparse_to_dense

In [ ]:
def dense_to_sparse_vector(vec):
    indices = []
    values = []
    for i, v in enumerate(vec):
        if v != 0:
            indices.append(i)
            values.append(v)
    return indices, values

def sparse_to_dense_vector(indices, values, n):
    vec = [0] * n
    for i, v in zip(indices, values):
        vec[i] = v
    return vec

# пример
v_dense = [2, 0, 3]  # размер 3

idx, vals = dense_to_sparse_vector(v_dense)
print("Sparse vector (из dense):")
print("indices =", idx)
print("values  =", vals)

v_back = sparse_to_dense_vector(idx, vals, len(v_dense))
print("Восстановленный dense-вектор:")
print(v_back)


Sparse vector (из dense):
indices = [0, 2]
values  = [2, 3]
Восстановленный dense-вектор:
[2, 0, 3]




---



#sparse vector × sparse matrix (CSR)

result = v * A

A = CSR (values, col_index, row_ptr)

result[j] = sum( по i: v[i] * A[i,j] )


In [ ]:
def multiply_sparse_vector_csr(indices, values,
                               csr_values, csr_col_index, csr_row_ptr,
                               n_cols):
    result = [0] * n_cols

    for k in range(len(indices)):
        row = indices[k]
        v_val = values[k]

        start = csr_row_ptr[row]
        end = csr_row_ptr[row + 1]

        for p in range(start, end):
            col = csr_col_index[p]
            a_val = csr_values[p]
            result[col] += v_val * a_val

    return result

# A =
# [0 0 5
#  7 0 0
#  0 0 1]

csr_values, csr_cols, csr_row_ptr = dense_to_csr(A)

# v:
v_dense = [2, 0, 3]
idx, vals = dense_to_sparse_vector(v_dense)

print("v (dense) =", v_dense)
print("v (sparse): indices =", idx, ", values =", vals)

res = multiply_sparse_vector_csr(idx, vals,
                                 csr_values, csr_cols, csr_row_ptr,
                                 n_cols=3)
print("Результат v * A:")
print(res)


v (dense) = [2, 0, 3]
v (sparse): indices = [0, 2] , values = [2, 3]
Результат v * A:
[0, 0, 13]




---



# Mini project: linear regression with csr

Этот проект решает задачу линейной регрессии с использованием как плотных (dense), так и разреженных (sparse) матриц. Генерируются данные для регрессии с разреженной матрицей признаков, после чего выполняются два метода линейной регрессии: один для плотных матриц, другой для разреженных (CSR). Сравниваются время выполнения и точность модели в зависимости от типа матрицы, что позволяет оценить эффективность работы с разреженными данными

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix
import time

# 1. generation

def generate_sparse_regression_data(n_samples=10000,
                                    n_features=2000,
                                    density=0.01,
                                    noise_std=0.1,
                                    random_state=42):
    rng = np.random.default_rng(random_state)

    # gennerating coo and convert to csr
    nnz_per_sample = int(n_features * density)
    if nnz_per_sample == 0:
        nnz_per_sample = 1

    data = []
    rows = []
    cols = []

    for i in range(n_samples):
        feat_idx = rng.choice(n_features, size=nnz_per_sample, replace=False)
        feat_val = rng.normal(loc=0.0, scale=1.0, size=nnz_per_sample)

        rows.extend([i] * nnz_per_sample)
        cols.extend(feat_idx.tolist())
        data.extend(feat_val.tolist())

    X_sparse = csr_matrix((data, (rows, cols)), shape=(n_samples, n_features))

    w_true = rng.normal(loc=0.0, scale=1.0, size=n_features)

    # y = X w_true + noise
    y = X_sparse @ w_true
    y = y + rng.normal(loc=0.0, scale=noise_std, size=n_samples)

    return X_sparse, y, w_true


# 2. linear regression - only dense

def linear_regression_gd_dense(X, y, lr=0.01, n_epochs=20, verbose=False):
    n_samples, n_features = X.shape
    w = np.zeros(n_features)

    for epoch in range(n_epochs):
        # pred
        y_pred = X @ w
        # mse
        grad = (2.0 / n_samples) * X.T @ (y_pred - y)
        # updating
        w -= lr * grad

        if verbose:
            mse = np.mean((y_pred - y) ** 2)
            print(f"[DENSE] epoch {epoch+1}, MSE = {mse:.4f}")

    return w


# 3. linear regression - only csr

def linear_regression_gd_sparse(X_csr, y, lr=0.01, n_epochs=20, verbose=False):
    n_samples, n_features = X_csr.shape
    w = np.zeros(n_features)

    for epoch in range(n_epochs):
        y_pred = X_csr @ w
        grad = (2.0 / n_samples) * (X_csr.T @ (y_pred - y))
        w -= lr * grad

        if verbose:
            mse = np.mean((y_pred - y) ** 2)
            print(f"[SPARSE] epoch {epoch+1}, MSE = {mse:.4f}")

    return w



# 4. results and comrapissoin

def main():
    print("Генерация данных...")
    X_sparse, y, w_true = generate_sparse_regression_data(
        n_samples=10000,
        n_features=2000,
        density=0.01,
        noise_std=0.1,
        random_state=42
    )

    print("Размер X:", X_sparse.shape)
    print("Количество ненулевых элементов:", X_sparse.nnz)
    total_elements = X_sparse.shape[0] * X_sparse.shape[1]
    print("Плотность:", X_sparse.nnz / total_elements)

    print("\n== DENSE ==")
    t0 = time.perf_counter()
    X_dense = X_sparse.toarray()
    t1 = time.perf_counter()
    print(f"Время конвертации sparse → dense: {t1 - t0:.4f} сек")

    t2 = time.perf_counter()
    w_dense = linear_regression_gd_dense(X_dense, y, lr=0.05, n_epochs=20)
    t3 = time.perf_counter()
    print(f"Время обучения (dense): {t3 - t2:.4f} сек")

    mse_dense = np.mean((X_dense @ w_dense - y) ** 2)
    print(f"MSE после обучения (dense): {mse_dense:.4f}")

    print("\n== SPARSE (CSR) ==")
    t4 = time.perf_counter()
    w_sparse = linear_regression_gd_sparse(X_sparse, y, lr=0.05, n_epochs=20)
    t5 = time.perf_counter()
    print(f"Время обучения (sparse CSR): {t5 - t4:.4f} сек")

    y_pred_sparse = X_sparse @ w_sparse
    mse_sparse = np.mean((y_pred_sparse - y) ** 2)
    print(f"MSE после обучения (sparse): {mse_sparse:.4f}")

    print("\n== Сравнение с w_true ==")
    diff_dense = np.linalg.norm(w_dense - w_true) / np.linalg.norm(w_true)
    diff_sparse = np.linalg.norm(w_sparse - w_true) / np.linalg.norm(w_true)
    print(f"Относительная ошибка w (dense):  {diff_dense:.4f}")
    print(f"Относительная ошибка w (sparse): {diff_sparse:.4f}")

if __name__ == "__main__":
    main()


Генерация данных...
Размер X: (10000, 2000)
Количество ненулевых элементов: 200000
Плотность: 0.01

== DENSE ==
Время конвертации sparse → dense: 0.1481 сек
Время обучения (dense): 1.6442 сек
MSE после обучения (dense): 18.4106

== SPARSE (CSR) ==
Время обучения (sparse CSR): 0.0284 сек
MSE после обучения (sparse): 18.4106

== Сравнение с w_true ==
Относительная ошибка w (dense):  0.9802
Относительная ошибка w (sparse): 0.9802
